In [1]:
import pandas as pd 

## read data

In [2]:
#Enter the name of the restaurant excel file
excel_sheet_name = "excel sheet 1.xlsx"

#Enter the database excel file name here
database_excel_name = "Recipes database.xlsx"

#enter the name of the excel file result
result_excel = "./excel sheet 2.xlsx"

In [3]:
def import_data(excel_sheet_name):
    xls_file = pd.ExcelFile(excel_sheet_name)
    sheets = xls_file.sheet_names
    sides = pd.DataFrame()
    desserts = pd.DataFrame()
    single_items = pd.read_excel(excel_sheet_name,sheet_name='single items')
    if 'sides' in sheets:
        sides = pd.read_excel(excel_sheet_name,sheet_name='sides')
    if 'drinks-desserts' in sheets:
        desserts = pd.read_excel(excel_sheet_name,sheet_name='drinks-desserts')
    return single_items,sides,desserts

In [4]:
single_items,sides,desserts = import_data(excel_sheet_name)
db_single_items,db_sides,db_desserts = import_data(database_excel_name)

In [5]:
print(db_single_items.shape[0])

357


In [6]:
print(db_desserts.shape[0])

38


In [7]:
(db_sides.shape[0])

46

# matching items' names

In [8]:
def common_words_number(base_item_words,research_item_words):
    words_founded_number =0
    for word in research_item_words: 
        if (word in base_item_words):
            words_founded_number +=1
    return(words_founded_number)  

In [9]:
def matching_names(data,database):
    matchings=[]
    for i,row in data.iterrows():
        item = data['menu'][i].lower()
        words=item.split()

        for j,line in database.iterrows():
            database_item = database['menu'][j].lower()
            database_item_words=database_item.split()
            if 'or' in database_item_words :
                database_item_words.remove('or')
            if 'and' in database_item_words :
                database_item_words.remove('and')
            words_founded_number_in_database = common_words_number(database_item_words,words)
            if words == database_item_words:
                data['Recipe Cost'][i] = database['Recipe Cost'][j]
                data['Research links'][i] = "Matched with "+database_item +"\nFounded in the recipe cost database at the line " + str(j+2)
                data['Meal recipe'][i] = database['Meal recipe'][j]
                
                
            elif ((len(words)==1) & (len(database_item_words)>1)) | ((len(database_item_words) ==1) & (len(words)>1)): 
                if (words_founded_number_in_database >= 1):
                    data['Recipe Cost'][i] = database['Recipe Cost'][j]
                    data['Research links'][i] = "Matched with "+database_item +"\nFounded in the recipe cost database at the line " + str(j+2)
                    data['Meal recipe'][i] = database['Meal recipe'][j]
                    
                
            else:
                    if words_founded_number_in_database>=2:
                        data['Recipe Cost'][i] = database['Recipe Cost'][j]
                        data['Research links'][i] = "Matched with "+database_item +"\nFounded in the recipe cost database at the line " + str(j+2)
                        data['Meal recipe'][i] = database['Meal recipe'][j]
                        
        if (data['Recipe Cost'][i]!=0):
            matchings.append(data['menu'][i])
    return data,matchings

In [10]:
single_items,single_matchings = matching_names(single_items,db_single_items)
if len(sides)!=0:
    sides,sides_matchings = matching_names(sides,db_sides)
if len(desserts)!=0:
    desserts,desserts_matchings = matching_names(desserts,db_desserts)

C:\Users\dridi\AppData\Local\Temp/ipykernel_19628/1894637490.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Recipe Cost'][i] = database['Recipe Cost'][j]
C:\Users\dridi\AppData\Local\Temp/ipykernel_19628/1894637490.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Research links'][i] = "Matched with "+database_item +"\nFounded in the recipe cost database at the line " + str(j+2)
C:\Users\dridi\AppData\Local\Temp/ipykernel_19628/1894637490.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

# save results

In [11]:
writer = pd.ExcelWriter(result_excel, engine = 'xlsxwriter')
single_items.to_excel(writer, sheet_name = 'single items',index=False)
if len(sides)!=0:
    sides.to_excel(writer, sheet_name = 'sides',index=False)
if len(desserts)!=0:
    desserts.to_excel(writer, sheet_name = 'drinks-desserts',index=False)
writer.save()